In [1]:
import config

import numpy as np
from tqdm import tqdm

import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)

In [20]:
mnist = tf.contrib.learn.datasets.mnist.load_mnist(train_dir="mnist_data")

Extracting mnist_data/train-images-idx3-ubyte.gz
Extracting mnist_data/train-labels-idx1-ubyte.gz
Extracting mnist_data/t10k-images-idx3-ubyte.gz
Extracting mnist_data/t10k-labels-idx1-ubyte.gz


In [3]:
mnist.train.images[mnist.train.images < 0.5] = 0
mnist.train.images[mnist.train.images > 0] = 1

In [4]:
ds = tf.contrib.distributions
xav_init = tf.contrib.layers.xavier_initializer()

In [5]:
def encoder(X, hidden_size):
    h_encoders = [
        tf.layers.dense(
            X,
            hidden_size[0],
            activation=tf.nn.relu,
            kernel_initializer=xav_init,
            name="encoder_hidden_layer_0"
        )
    ]
    
    for index, size in enumerate(hidden_size[1:]):
        h_encoders.append(
            tf.layers.dense(
                h_encoders[index],
                size,
                activation=tf.nn.relu,
                kernel_initializer=xav_init,
                name="encoder_hidden_layer_" + str(index + 1)
            )
        )
    
    encoder_mean = tf.layers.dense(
        h_encoders[-1],
        config.latent_dim,
        kernel_initializer=xav_init,
        name="encoder_mean"
    )
    encoder_log_var = tf.layers.dense(
        h_encoders[-1],
        config.latent_dim,
        kernel_initializer=xav_init,
        name="encoder_log_variance"
    )
    
    return encoder_mean, encoder_log_var

In [6]:
def decoder(Z, hidden_size):
    h_decoders = [
        tf.layers.dense(
            Z,
            hidden_size[0],
            activation=tf.nn.relu,
            kernel_initializer=xav_init,
            name="decoder_hidden_layer_0"
        )
    ]
    
    for index, size in enumerate(hidden_size[1:]):
        h_decoders.append(
            tf.layers.dense(
                h_decoders[index],
                size,
                activation=tf.nn.relu,
                kernel_initializer=xav_init,
                name="decoder_hidden_layer_" + str(index + 1)
            )
        )
    
    out_X = tf.layers.dense(
        h_decoders[-1],
        config.input_dim,
        kernel_initializer=xav_init,
        name="decoder_X"
    )
    
    return out_X

In [7]:
def sample_Z(encoder_mean, encoder_log_var, epsilon):
    return encoder_mean + tf.exp(encoder_log_var / 2) * epsilon

In [8]:
def init_prior():
    prior_means = tf.Variable(
        tf.zeros((config.n_clusters, config.latent_dim)),
        dtype=tf.float32,
        trainable=False,
        name="prior_means"
    )
    prior_vars = tf.Variable(
        tf.ones((config.n_clusters, config.latent_dim)),
        dtype=tf.float32,
        trainable=False,
        name="prior_vars"
    )
    prior_weights = tf.Variable(
        tf.ones((config.n_clusters)) / config.n_clusters,
        dtype=tf.float32,
        trainable=False,
        name="prior_weights"
    )
    
    return tf.Variable(prior_means), tf.Variable(prior_vars), tf.Variable(prior_weights)

In [9]:
def q_c(Z):
    def fn_cluster(_, k):
        q = prior_weights[k] * ds.MultivariateNormalDiag(loc=prior_means[k], scale_diag=prior_vars[k]).prob(Z)
        return tf.reshape(q, [config.batch_size])

    clusters = tf.Variable(tf.range(config.n_clusters))
    probs = tf.scan(fn_cluster, clusters, initializer=tf.ones([config.batch_size]))
    probs = tf.transpose(probs)
    probs = probs / tf.reshape(tf.reduce_sum(probs, 1), (-1, 1))
    return probs

In [10]:
def vae_loss(cluster_weights, decoded_X, encoder_mean, encoder_log_var):
#     J = tf.reduce_sum(tf.nn.sigmoid_cross_entropy_with_logits(labels=X, logits=decoded_X), axis=1)
    J = 0.0
    J -= tf.reduce_sum(X * tf.log(tf.sigmoid(decoded_X)), axis=1)
    J -= tf.reduce_sum((1 - X) * tf.log(1 - tf.sigmoid(decoded_X)), axis=1)
    J -= tf.reduce_sum(cluster_weights * tf.log(prior_weights), axis=1)
    J += tf.reduce_sum(cluster_weights * tf.log(cluster_weights), axis=1)
    J -= 0.5 * tf.reduce_sum(1 + encoder_log_var, axis=1)

    def loop_cluster(previous_output, current_input):
        k = current_input
        l = previous_output + 0.5 * cluster_weights[:, k] * tf.reduce_sum(
            tf.log(prior_vars[k]) + (tf.exp(encoder_log_var) + tf.square(encoder_mean - prior_means[k])) / prior_vars[k], axis=1
        )
        return l

    clusters = tf.Variable(tf.range(config.n_clusters))
    y = tf.scan(loop_cluster, clusters, initializer=tf.zeros(config.batch_size))
    
    J += y[-1, :]
    
    return tf.reduce_mean(J)

In [11]:
def update_prior_weights_f(prior_weights, cluster_weights):
    return tf.reduce_sum(cluster_weights, axis=0) / float(config.batch_size)

def update_prior_means_f(prior_means, encoder_mean, cluster_weights):
    def loop_cluster(_, k):
        t = tf.matmul(tf.reshape(cluster_weights[:, k], [1, config.batch_size]), encoder_mean)
        t = tf.reshape(t, [config.latent_dim]) / tf.reduce_sum(cluster_weights[:, k], axis=0)
        return t

    clusters = tf.Variable(tf.range(config.n_clusters))
    return tf.scan(loop_cluster, clusters, initializer=tf.ones([config.latent_dim]))

def update_prior_vars_f(prior_weights, encoder_mean, encoder_log_var, cluster_weights):
    global prior_means
    
    s = tf.matmul(
        tf.reshape(cluster_weights[:, 0], [1, config.batch_size]),
        (tf.exp(encoder_log_var) + tf.square(encoder_mean - prior_means[0]))
    ) / tf.reduce_sum(cluster_weights[:, 0], axis=0)
    
    for i in xrange(1, config.n_clusters):
        t = tf.matmul(
            tf.reshape(cluster_weights[:, i], [1, config.batch_size]),
            tf.exp(encoder_log_var) + tf.square(encoder_mean - prior_means[i])
        ) / tf.reduce_sum(cluster_weights[:, i], axis=0)
        s = tf.concat([s, t], 0)
        
    return s

In [12]:
epoch_len = len(mnist.train.images) / config.batch_size
decay_steps = epoch_len * 10.0

In [13]:
X = tf.placeholder(tf.float32, [None, config.input_dim])
epsilon = tf.placeholder(tf.float32, [None, config.latent_dim])

prior_means, prior_vars, prior_weights = init_prior()

encoder_mean, encoder_log_var = encoder(X, config.encoder_hidden_size)

Z = sample_Z(encoder_mean, encoder_log_var, epsilon)

decoded_X = decoder(Z, config.decoder_hidden_size)

cluster_weights = q_c(Z)

In [14]:
loss = vae_loss(cluster_weights, decoded_X, encoder_mean, encoder_log_var)

In [15]:
update_prior_weights = prior_weights.assign(update_prior_weights_f(prior_weights, cluster_weights))
update_prior_means = prior_means.assign(update_prior_means_f(prior_means, encoder_mean, cluster_weights))
update_prior_vars = prior_vars.assign(update_prior_vars_f(prior_vars, encoder_mean, encoder_log_var, cluster_weights))

In [16]:
# train_step = tf.train.AdamOptimizer(config.adam_nn_learning_rate, epsilon=config.adam_nn_epsilon).minimize(loss)

learning_rate = tf.train.exponential_decay(.0001, 0, decay_steps, 0.9, staircase=True)
train_step = tf.train.AdamOptimizer(learning_rate).minimize(loss)

In [17]:
sess = tf.Session()
tf.global_variables_initializer().run(session=sess)

In [18]:
for epoch in range(config.n_epochs):
    J = 0.0
    for i in tqdm(range(epoch_len)):
        X_batch = mnist.train.next_batch(config.batch_size)[0]
        out = sess.run(
            [loss, train_step, decoded_X],
            feed_dict={
                X: X_batch,
                epsilon: np.random.randn(config.batch_size, config.latent_dim)
            }
        )
        J += out[0] / epoch_len
        print out[2]

  1%|          | 4/550 [00:00<01:21,  6.70it/s]

[[ 3.7619643e-02 -2.5851820e-03  2.1526869e-02 ... -3.1456128e-02
  -9.2494078e-03  3.8292110e-02]
 [ 7.7667707e-03  1.9597573e-02  2.9862779e-03 ... -2.8807895e-02
  -7.7653844e-03  2.8161557e-02]
 [ 5.3861413e-02  2.1212503e-02  8.7726414e-03 ... -9.0193935e-06
   1.2380347e-02  2.0190638e-02]
 ...
 [ 9.8441519e-02 -1.6560830e-02 -4.2007875e-02 ... -6.3983083e-02
  -4.0526588e-02  1.2515545e-01]
 [ 6.3162006e-02 -1.8229317e-03  1.4642775e-02 ... -5.9018899e-02
   2.6496463e-02  8.6429883e-03]
 [ 3.7428990e-02 -2.1097194e-03 -3.7908249e-02 ... -3.9442528e-02
  -3.4530733e-02  7.3456153e-02]]
[[ 0.04900484 -0.02242715 -0.01281662 ... -0.02618276 -0.05104877
   0.02793771]
 [ 0.04197288  0.00811505 -0.03426424 ... -0.03314707  0.01363808
   0.04945255]
 [ 0.06248932 -0.00311447 -0.05319267 ... -0.02194583  0.0176005
   0.00436698]
 ...
 [ 0.03390752 -0.01671645 -0.01614124 ... -0.01599197 -0.0342388
  -0.00096665]
 [ 0.02500686  0.00387764 -0.00744782 ... -0.03656271  0.00496347
   0.02

  2%|▏         | 10/550 [00:00<00:46, 11.72it/s]

[[ 0.11554861 -0.06110954 -0.05266365 ... -0.1170681   0.03686757
   0.01737048]
 [ 0.03697248 -0.03832969 -0.01878012 ... -0.04252649 -0.00071669
   0.05076703]
 [ 0.12588245 -0.0533418  -0.03071393 ... -0.07439367 -0.00043667
   0.04014741]
 ...
 [ 0.11577027 -0.09017895 -0.07454616 ... -0.07407776 -0.0238529
   0.07001716]
 [ 0.09711653 -0.06833935 -0.03995633 ... -0.07741334  0.03005173
  -0.04518557]
 [ 0.04393023 -0.05590809 -0.07162232 ... -0.06055573 -0.04683796
   0.05164234]]
[[ 0.04254043 -0.07325853 -0.04641149 ... -0.04840251  0.00195605
   0.03550543]
 [ 0.06952185 -0.0584863  -0.05178742 ... -0.08278862  0.00768973
   0.04240479]
 [ 0.09042967 -0.04610316 -0.08851577 ... -0.07296596 -0.00614174
   0.01642446]
 ...
 [ 0.09828959 -0.10058074 -0.08962849 ... -0.06613969  0.03096485
   0.04837882]
 [ 0.09648207 -0.06890529 -0.08333847 ... -0.08113129 -0.09704649
   0.06600399]
 [ 0.07948285 -0.12464511 -0.07223947 ... -0.07354818  0.04512856
   0.03528117]]
[[ 0.07864735 -0.

  3%|▎         | 16/550 [00:01<00:37, 14.38it/s]

...

 28%|██▊       | 153/550 [00:08<00:21, 18.67it/s]

[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]


 29%|██▉       | 159/550 [00:08<00:20, 18.84it/s]

[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]


KeyboardInterrupt: 